In [7]:
#Peri, Luke, and Ryan: Shor's Algorithm
#Using code from https://github.com/ttlion/ShorAlgQiskit

#All imports
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, IBMQ, BasicAer
import sys, math, array, fractions
import numpy as np

#QFT implementation, from https://github.com/ttlion/ShorAlgQiskit
def create_QFT(circuit,up_reg,n):
    i=n-1
    #Apply the H gates and Cphases
    while i>=0:
        circuit.h(up_reg[i])        
        j=i-1  
        while j>=0:
            if (np.pi)/(pow(2,(i-j))) > 0:
                circuit.cp( (np.pi)/(pow(2,(i-j))) , up_reg[i] , up_reg[j] )
                j=j-1   
        i=i-1  

#Inverse QFT implementation, from https://github.com/ttlion/ShorAlgQiskit
def create_inverse_QFT(circuit,up_reg,n,with_swaps):
    #If specified, apply the Swaps at the beggining
    if with_swaps==1:
        i=0
        while i < ((n-1)/2):
            circuit.swap(up_reg[i], up_reg[n-1-i])
            i=i+1
    #Apply the H gates and Cphases
    i=0
    while i<n:
        circuit.h(up_reg[i])
        if i != n-1:
            j=i+1
            y=i
            while y>=0:
                 if (np.pi)/(pow(2,(j-y))) > 0:
                    circuit.cp( - (np.pi)/(pow(2,(j-y))) , up_reg[j] , up_reg[y] )
                    y=y-1   
        i=i+1
    
#Continued fraction algorithm, modified from https://github.com/ttlion/ShorAlgQiskit
def get_factors(x_value,t_upper,N,a):
    if x_value<=0:
        print('-> x_value is <= 0, there is no continued fraction.')
        return False
    #Calculate T and x/T
    T = pow(2,t_upper)
    x_over_T = x_value/T
    #Cycle in which each iteration corresponds to putting one more term in the
    #calculation of the Continued Fraction (CF) of x/T 
    i=0
    b = array.array('i')
    t = array.array('f')
    b.append(math.floor(x_over_T))
    t.append(x_over_T - b[i])
    while i>=0:
        #From the 2nd iteration onwards, calculate the new terms of the CF based on the previous terms
        if i>0:
            b.append( math.floor( 1 / (t[i-1]) ) ) 
            t.append( ( 1 / (t[i-1]) ) - b[i] )
        #Calculate the continued fraction
        aux = 0
        j=i
        while j>0:    
            aux = 1 / ( b[j] + aux )      
            j = j-1
        aux = aux + b[0]
        #Get the denominator from the value obtained
        frac = fractions.Fraction(aux).limit_denominator()
        den=frac.denominator
        print("-> Numerator:{0}, Denominator: {1}".format(frac.numerator,frac.denominator))
        i=i+1
        if (den%2) == 1:
            if i>=15:
                print('-> The process failed to calculate the continued fraction, too many iterations.')
                return False
            continue
        #If denominator even, try to get factors of N
        try:
            exponential=pow(a ,(den/2))
        except:
            print('-> The process failed to calculate the continued fraction, denominator too large.')
        putting_plus = int(exponential + 1)
        putting_minus = int(exponential - 1)
        one_factor = math.gcd(putting_plus,N)
        other_factor = math.gcd(putting_minus,N)
        if one_factor==1 or one_factor==N or other_factor==1 or other_factor==N:
            print('-> Found just trivial factors.')
            if t[i-1]==0:
                print('-> The continued fraction has terminated.')
                return False
            else:
                print('-> The process failed to calculate the continued fraction.')
                return False         
        else:
            print('!-> The factors of {0} are {1} and {2}'.format(N,one_factor,other_factor))
            return True
        
#Calculates the array of angles to be used in the addition in Fourier Space, from https://github.com/ttlion/ShorAlgQiskit
def getAngles(a,N):
    s=bin(int(a))[2:].zfill(N) 
    angles=np.zeros([N])
    for i in range(0, N):
        for j in range(i,N):
            if s[j]=='1':
                angles[N-i-1]+=math.pow(2, -(j-i))
        angles[N-i-1]*=np.pi
    return angles

#Double controlled phase gate, from https://github.com/ttlion/ShorAlgQiskit
def ccphase(circuit,angle,ctl1,ctl2,tgt):
    circuit.cp(angle/2,ctl1,tgt)
    circuit.cx(ctl2,ctl1)
    circuit.cp(-angle/2,ctl1,tgt)
    circuit.cx(ctl2,ctl1)
    circuit.cp(angle/2,ctl2,tgt)

#Addition in Fourier Space, from https://github.com/ttlion/ShorAlgQiskit
def phiADD(circuit,q,a,N,inv):
    angle=getAngles(a,N)
    for i in range(0,N):
        if inv==0:
            circuit.p(angle[i],q[i])
        else:
            circuit.p(-angle[i],q[i])

#Single controlled version of the phiADD circuit, from https://github.com/ttlion/ShorAlgQiskit
def cphiADD(circuit,q,ctl,a,n,inv):
    angle=getAngles(a,n)
    for i in range(0,n):
        if inv==0:
            circuit.cp(angle[i],ctl,q[i])
        else:
            circuit.cp(-angle[i],ctl,q[i])

#Doubly controlled version of the phiADD circuit, from https://github.com/ttlion/ShorAlgQiskit    
def ccphiADD(circuit,q,ctl1,ctl2,a,n,inv):
    angle=getAngles(a,n)
    for i in range(0,n):
        if inv==0:
            ccphase(circuit,angle[i],ctl1,ctl2,q[i])
        else:
            ccphase(circuit,-angle[i],ctl1,ctl2,q[i])
        
#Circuit that implements doubly controlled modular addition by a, from https://github.com/ttlion/ShorAlgQiskit
def ccphiADDmodN(circuit, q, ctl1, ctl2, aux, a, N, n):
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)
    phiADD(circuit, q, N, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.cx(q[n-1],aux)
    create_QFT(circuit,q,n)
    cphiADD(circuit, q, aux, N, n, 0)
    
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.x(q[n-1])
    circuit.cx(q[n-1], aux)
    circuit.x(q[n-1])
    create_QFT(circuit,q,n)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)

#Circuit that implements the inverse of doubly controlled modular addition by a, from https://github.com/ttlion/ShorAlgQiskit
def ccphiADDmodN_inv(circuit, q, ctl1, ctl2, aux, a, N, n):
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.x(q[n-1])
    circuit.cx(q[n-1],aux)
    circuit.x(q[n-1])
    create_QFT(circuit, q, n)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)
    cphiADD(circuit, q, aux, N, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.cx(q[n-1], aux)
    create_QFT(circuit, q, n)
    phiADD(circuit, q, N, n, 0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)

#Circuit that implements single controlled modular multiplication by a, from https://github.com/ttlion/ShorAlgQiskit
def cMULTmodN(circuit, ctl, q, aux, a, N, n):
    create_QFT(circuit,aux,n+1)
    for i in range(0, n):
        ccphiADDmodN(circuit, aux, q[i], ctl, aux[n+1], (2**i)*a % N, N, n+1)
    create_inverse_QFT(circuit, aux, n+1, 0)

    for i in range(0, n):
        circuit.cswap(ctl,q[i],aux[i])

    a_inv = modinv(a, N)
    create_QFT(circuit, aux, n+1)
    i = n-1
    while i >= 0:
        ccphiADDmodN_inv(circuit, aux, q[i], ctl, aux[n+1], math.pow(2,i)*a_inv % N, N, n+1)
        i -= 1
    create_inverse_QFT(circuit, aux, n+1, 0)

#GCD function
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

#Modular inverse function
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m    

    
#PROGRAM STARS HERE
if __name__ == '__main__':
    
    #SET CONSTANTS
    N=15
    a=7
    number_shots=10
    
    #Print info to user
    print('Executing the circuit {0} times for N={1} and a={2}'.format(number_shots,N,a))
    #Calculate number of qubits
    n = math.ceil(math.log(N,2))
    print('Total number of qubits used: {0}'.format(4*n+2))
    #Create registers
    aux = QuantumRegister(n+2) #auxilliary quantum register used in addition and multiplication
    up_reg = QuantumRegister(2*n) #quantum register where the sequential QFT is performed
    down_reg = QuantumRegister(n) #quantum register where the multiplications are made"
    up_classic = ClassicalRegister(2*n) #classical register where the measured values of the QFT are stored
    #Create Quantum Circuit
    circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic)
    #Initialize down register to 1 and create maximal superposition in top register
    circuit.h(up_reg)
    circuit.x(down_reg[0])
    #Apply the multiplication gates
    for i in range(0, 2*n):
        cMULTmodN(circuit, up_reg[i], down_reg, aux, int(pow(a, pow(2, i))), N, n)
    #Apply inverse QFT
    create_inverse_QFT(circuit, up_reg, 2*n ,1)
    #Measure the top qubits, to get x value
    circuit.measure(up_reg,up_classic)
    #Simulate the created Quantum Circuit
    simulation = execute(circuit, backend=BasicAer.get_backend('qasm_simulator'),shots=number_shots)
    #Get the results of the simulation in proper structure
    sim_result=simulation.result()
    counts_result = sim_result.get_counts(circuit)
    print("Circuit run successfully for {0} times!".format(number_shots))
    #Print info to user from the simulation results
    i=0
    while i < len(counts_result):
        print('-> Result \"{0}\" happened {1} times out of {2}'.format(list(sim_result.get_counts().keys())[i],list(sim_result.get_counts().values())[i],number_shots))
        i=i+1
    #For each simulation result, print proper info to user and try to calculate the factors of N
    prob_success=0
    i=0
    while i < len(counts_result):
        #Get the x_value from the final state qubits
        output_desired = list(sim_result.get_counts().keys())[i]
        x_value = int(output_desired, 2)
        prob_this_result = 100 * ( int( list(sim_result.get_counts().values())[i] ) ) / (number_shots)
        print("Result {0} happened in {1:.4f}% of all cases".format(output_desired,prob_this_result))
        #Print the final x_value to user
        print('-> This result is {0} in decimal.'.format(x_value))
        #Get the factors using the x value obtained
        success=get_factors(int(x_value),int(2*n),int(N),int(a))
        if success==True:
            prob_success = prob_success + prob_this_result
        i=i+1
    print("\nUsing a={0}, found the factors of N={1} in {2:.4f} % of the cases\n".format(a,N,prob_success))

Executing the circuit 10 times for N=15 and a=7
Total number of qubits used: 18
Circuit run successfully for 10 times!
-> Result "10000000" happened 3 times out of 10
-> Result "01000000" happened 1 times out of 10
-> Result "00000000" happened 4 times out of 10
-> Result "11000000" happened 2 times out of 10
Result 10000000 happened in 30.0000% of all cases
-> This result is 128 in decimal.
-> Numerator:0, Denominator: 1
-> Numerator:1, Denominator: 2
-> Found just trivial factors.
-> The continued fraction has terminated.
Result 01000000 happened in 10.0000% of all cases
-> This result is 64 in decimal.
-> Numerator:0, Denominator: 1
-> Numerator:1, Denominator: 4
!-> The factors of 15 are 5 and 3
Result 00000000 happened in 40.0000% of all cases
-> This result is 0 in decimal.
-> x_value is <= 0, there is no continued fraction.
Result 11000000 happened in 20.0000% of all cases
-> This result is 192 in decimal.
-> Numerator:0, Denominator: 1
-> Numerator:1, Denominator: 1
-> Numerato